<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

We need to create a new csv file containing unique restaurants from inspections let's say **2017->2019** (don't forget to filter out "out of business"...) with a bunch of useful features and computed risk score.



In [1]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

In [57]:
restaurants_corpus = pd.DataFrame(columns = ['restaurant licence','latitude',\
                                             'longitude','community area',\
                                             'restaurant name','rate of failed inspections',\
                                             'rate of successful inspections',\
                                             'Average nbr of important violations',\
                                             'Food chain flag', \
                                             'nbr risk_related words computed from inspectors comments',\
                                              'Risk_score'])


In [58]:
restaurants_corpus

,restaurant licence,latitude,longitude,community area,restaurant name,rate of failed inspections,rate of successful inspections,Average nbr of important violations,Food chain flag,nbr risk_related words computed from inspectors comments,Risk_score


### TO DO: 

<font color='red'>
Read yelp dataset and chain dataset to add wanted features

Then calculate below 
'rate of failed inspections',\
'rate of successful inspections',\
'Average nbr of important violations',\
'Food chain flag', \
'nbr risk_related words computed from inspectors comments',\
'Risk_score'</font>

In [4]:
#Read the necessary cleaned datasets to keep the selected features
cfi_dataset = pd.read_pickle('datasets/cleaned_inspections.pickle')

In [5]:
cfi_dataset_selected = cfi_dataset[['Inspection ID', 'DBA Name', 'AKA Name', 'License #', 'Latitude', 'Longitude', 
                                    'Location', 'Community Area', 'Inspection Date', 'Results', 'Risk', 
                                    'Violation Numbers', 'Violation Comments']].copy()

Since we examined earlier in the Milestone2, as shown below 
<img src="datasets/countresult.png" style="height:200px"> 

here we keep only the data with "Pass", "Fail", "Pass w/ Conditions" results.


In [6]:
# Keep only the wanted results from data 
cfi_dataset_selected = cfi_dataset_selected[cfi_dataset_selected['Results'].isin(['Pass', "Fail", "Pass w/ Conditions"])]

Now let's eliminate the violations with NaN values from the data because our risk score analysis will be based on violations.

In [7]:
cfi_dataset_selected = cfi_dataset_selected[~pd.isnull(cfi_dataset_selected['Violation Numbers'])]

In [8]:
cfi_dataset_selected = cfi_dataset_selected.reset_index()

In [9]:
cfi_dataset_selected = cfi_dataset_selected.drop(columns='index')

In [10]:
# Total number of violation counts
cfi_dataset_selected['Violation Counts'] = [len(cfi_dataset_selected['Violation Numbers'][i]) for i in range(len(cfi_dataset_selected))]


In [50]:
# Total number of violation counts according to some gradings 
# source: https://www1.nyc.gov/assets/doh/downloads/pdf/rii/how-we-score-grade.pdf
'''
count the violation numbers for each inspection:
if there is violation number between 0 and 13 points it will be counted as 'Critical Violations Count'
if there is violation number between 14 and 27 points it will be counted as 'Moderate Violations Count'
if there is violation number between 28 and more points it will be counted as 'Non-Critical Violations Count'
'''

cfi_dataset_selected['Critical Violations Count'] = [int(len([ i for i in cfi_dataset_selected['Violation Numbers'][k] if i<14])) for k in range(len(cfi_dataset_selected))]
cfi_dataset_selected['Moderate Violations Count'] = [int(len([ i for i in cfi_dataset_selected['Violation Numbers'][k] if i>13 and i<28])) for k in range(len(cfi_dataset_selected))]
cfi_dataset_selected['Non-Critical Violations Count'] = [int(len([ i for i in cfi_dataset_selected['Violation Numbers'][k] if i>27])) for k in range(len(cfi_dataset_selected))]


In [22]:
# Adding Allergen flag to the dataset
cfi_dataset_selected['Allergen Flag'] = np.nan
for i in range(len(cfi_dataset_selected)):
    each_comment[i] = pd.Series([x.strip() for x in cfi_dataset_selected['Violation Comments'][i]])
    if (each_comment[i].astype(str).str.contains('ALLERGEN', flags=re.IGNORECASE, regex=True)== True).any():
        cfi_dataset_selected['Allergen Flag'][i] = 'Y'
    else: 
        cfi_dataset_selected['Allergen Flag'][i] = 'N'

In [23]:
# Adding Vomit or Diarrheal Events Flag to the dataset
cfi_dataset_selected['Vomit or Diarrheal Flag'] = np.nan
for i in range(len(cfi_dataset_selected)):
    each_comment[i] = pd.Series([x.strip() for x in cfi_dataset_selected['Violation Comments'][i]])
    if (each_comment[i].astype(str).str.contains('VOMIT', flags=re.IGNORECASE, regex=True)== True).any():
        cfi_dataset_selected['Vomit or Diarrheal Flag'][i] = 'Y'
    elif (each_comment[i].astype(str).str.contains('DIARRHEAL', flags=re.IGNORECASE, regex=True)== True).any():
        cfi_dataset_selected['Vomit or Diarrheal Flag'][i] = 'Y'
    else:
        cfi_dataset_selected['Vomit or Diarrheal Flag'][i] = 'N'

In [52]:
cfi_dataset_selected.head()

,Inspection ID,DBA Name,AKA Name,License #,Latitude,Longitude,Location,Community Area,Inspection Date,Results,Risk,Violation Numbers,Violation Comments,Violation Counts,Allergen Flag,Vomit or Diarrheal Flag,Critical Violations Count,Moderate Violations Count,Non-Critical Violations Count
0,2345318,SUBWAY,SUBWAY,2529116,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2019-11-08,Pass w/ Conditions,1,"[3, 5, 58]",[ 2-102.14(O) OBSERVED NO WRITTEN EMPLOYEE HE...,3,Y,Y,2,0,1
1,2321166,SUBWAY,SUBWAY,2529116,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2019-11-06,Fail,1,"[3, 5, 10, 22, 33, 57, 58]",[ OBSERVED NO WRITTEN EMPLOYEE HEALTH POLICY O...,7,Y,Y,3,1,3
2,2290717,PAPA JOHN'S PIZZA,PAPA JOHN'S PIZZA,1545897,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2019-05-30,Pass w/ Conditions,2,"[3, 5]",[ OBSERVED NO EMPLOYEE HEALTH POLICY ON THE PR...,2,N,Y,2,0,0
3,2144588,SUBWAY,SUBWAY,2529116,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2018-02-06,Fail,1,"[16, 34, 42]",[ OBSERVED BLACK AND PINK MOLD LIKE SUBSTANCES...,3,N,N,0,1,2
4,2064444,PAPA JOHN'S PIZZA,PAPA JOHN'S PIZZA,1545897,41.927995,-87.785752,"41.92799528871574, -87.78575236468352",Belmont Cragin,2017-06-16,Pass,2,"[32, 33, 34, 35, 37, 43, 45]","[ Inspector Comments, Inspector Comments, ...",7,N,N,0,0,7


In [53]:
#cfi_dataset_selected.to_pickle('pickles/cfi_dataset_selected')

In [55]:
cfi_dataset_selected_pickle = pd.read_pickle('pickles/cfi_dataset_selected')

In [61]:
cfi_dataset_selected_pickle.describe()

,Inspection ID,License #,Latitude,Longitude,Risk,Violation Counts,Critical Violations Count,Moderate Violations Count,Non-Critical Violations Count
count,1.182300e+05,1.182300e+05,118230.000000,118230.000000,118230.000000,118230.000000,118230.000000,118230.000000,118230.000000
mean,1.455131e+06,1.595582e+06,41.883736,-87.676102,1.295957,4.414971,0.419893,0.379421,3.615656
std,6.388562e+05,9.020764e+05,0.079277,0.059985,0.560864,2.910267,0.864290,0.699434,2.284933
min,4.424700e+04,0.000000e+00,41.644670,-87.914428,1.000000,1.000000,0.000000,0.000000,0.000000
25%,1.150865e+06,1.196827e+06,41.844671,-87.705859,1.000000,2.000000,0.000000,0.000000,2.000000
50%,1.497306e+06,1.979467e+06,41.892249,-87.664223,1.000000,4.000000,0.000000,0.000000,3.000000
75%,2.010082e+06,2.245453e+06,41.939869,-87.634293,1.000000,6.000000,0.000000,1.000000,5.000000
max,2.345339e+06,3.846407e+06,42.021064,-87.526940,3.000000,34.000000,9.000000,6.000000,23.000000


<font color='red'>
We can drop violation comments column right now, no more need.
    </font>

In [69]:
group_by_ = cfi_dataset_selected_pickle.groupby(by=['AKA Name', 'Results']).size().reset_index(name="Inspection Count")
group_by_ = group_by_.sort_values(by=['AKA Name','Results','Inspection Count'], ascending= False)
group_by_

,AKA Name,Results,Inspection Count
38200,tien giang restaurant,Fail,2
38199,the FRENCH LUNCHBOX,Pass w/ Conditions,1
38198,the FRENCH LUNCHBOX,Pass,2
38197,subway restaurant 1,Pass,1
38196,stockton,Pass,3
...,...,...,...
4,#1 DELI,Pass w/ Conditions,1
3,#1 CHOP SUEY,Pass w/ Conditions,5
2,#1 CHOP SUEY,Pass,20
1,#1 CHOP SUEY,Fail,10


And then we will compute the **Risk score** using a weighted sum of the features by their importance and use MinMax to normalize the score for each restaurant at the end :D